# Eat Safe, Love

## Notebook Set Up

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
collections = db.list_collection_names()
collections

['establishments']

In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [6]:
# Define the query to find establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments with a hygiene score of 20: {count}")

# Display the first document in the results using pprint
result = establishments.find_one(query)
pprint(result)

Number of establishments with a hygiene score of 20: 369
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': 0,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65639bd1d0e6c8b0510d043c'),
 'geocode': {'latitude': Decimal128('50.769705'),
             'longitude': Decimal128('0.27694')},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establi

In [7]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame([result])

# Display the number of rows in the DataFrame
num_rows = len(df)
print(f"Number of rows in the DataFrame: {num_rows}")

# Display the first 10 rows of the DataFrame with a prettier format
print("First 10 rows of the DataFrame:")
print(df.head(10).to_string(index=False))

Number of rows in the DataFrame: 1
First 10 rows of the DataFrame:
                     _id  FHRSID  ChangesByServerID LocalAuthorityBusinessID        BusinessName    BusinessType  BusinessTypeID         AddressLine1 AddressLine2 AddressLine3 AddressLine4 PostCode Phone  RatingValue    RatingKey          RatingDate LocalAuthorityCode LocalAuthorityName                    LocalAuthorityWebSite      LocalAuthorityEmailAddress                                                          scores SchemeType                                       geocode RightToReply    Distance  NewRatingPending                                                                                                                                                             meta                                                                              links
65639bd1d0e6c8b0510d043c  110681                  0                     4029 The Chase Rest Home Caring Premises               5 5-6 Southfields Road   Eastbourne  

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [8]:
# Find the establishments in London with a RatingValue greater than or equal to 4
query = {
    "LocalAuthorityName": "London",
    "RatingValue": {"$gte": 4}
}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
count

# Display the first document in the results using pprint
first_document = establishments.find_one(query)
pprint(first_document)

None


In [9]:
# Convert the list of dictionaries to a DataFrame
result_list = list(establishments.find(query))
df = pd.DataFrame(result_list)

# Display the number of rows in the DataFrame
num_rows = len(df)
num_rows

# Display the first 10 rows of the DataFrame
first_10_rows = df.head(10)
first_10_rows

""


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [10]:
# Define the latitude and longitude for "Penang Flavours"
latitude_pf = 51.490142
longitude_pf = 0.083840

# Search within 0.01 degree on either side of the latitude and longitude.
degree_search = 0.01

# Define the query to filter establishments with RatingValue equal to 5
query = {
    "RatingValue": 5,
    "geocode.latitude": {
        "$gte": latitude_pf - degree_search,
        "$lte": latitude_pf + degree_search
    },
    "geocode.longitude": {
        "$gte": longitude_pf - degree_search,
        "$lte": longitude_pf + degree_search
    }
}

# Sort by hygiene score in ascending order (1 means ascending, -1 means descending)
sort = [("scores.Hygiene", 1)]

# Find and fetch the results
results = establishments.find(query).sort(sort)

# Convert the results to a Pandas DataFrame
result_list = list(results)
df = pd.DataFrame(result_list)

# Print the results
df.head()

,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,LocalAuthorityWebSite,LocalAuthorityEmailAddress,scores,SchemeType,geocode,RightToReply,Distance,NewRatingPending,meta,links
0,65639bd8d0e6c8b0510d5c6d,1380578,0,14425,Howe and Co Fish and Chips - Van 17,Mobile caterer,7846,Restaurant And Premises 107A Plumstead High St...,,Plumstead,...,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': 0.0925370007753372, 'latitude': ...",,4646.955931,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
1,65639bd8d0e6c8b0510d5c8c,694609,0,PI/000116619,Volunteer,Pub/bar/nightclub,7843,130 - 132 Plumstead High Street,,Plumstead,...,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': 0.09208, 'latitude': 51.4873437}",,4646.965635,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
2,65639bd8d0e6c8b0510d5ca4,695223,0,PI/000178842,Iceland,Retailers - supermarkets/hypermarkets,7840,144 - 146 Plumstead High Street,,Plumstead,...,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': 0.0924199968576431, 'latitude': ...",,4646.946071,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
3,65639bd8d0e6c8b0510d5ca9,909912,0,PI/000201100,Lumbini Grocery Ltd T/A Al-Iman,Retailers - other,4613,17 Lakedale Road,,Plumstead,...,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': 0.0916256, 'latitude': 51.4871632}",,4646.975587,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
4,65639bd8d0e6c8b0510d5cac,694478,0,PI/000086506,Atlantic Fish Bar,Takeaway/sandwich shop,7844,35 Lakedale Road,,Plumstead,...,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': 0.0912164, 'latitude': 51.4867296}",,4646.974612,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."


In [12]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame

# Confirm the conversion
print("Result has been converted to a DataFrame.")

Result has been converted to a DataFrame.


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [13]:
# Create a MongoClient instance
client = MongoClient(port=27017)

# Access the 'uk_food' database
db = client['uk_food']

# Access the 'establishments' collection
establishments = db['establishments']

# Define the aggregation pipeline
pipeline = [
    {
        "$match": {
            "scores.Hygiene": 0
        }
    },
    {
        "$group": {
            "_id": "$LocalAuthorityName",
            "count": {"$sum": 1}
        }
    },
    {
        "$sort": {
            "count": -1
        }
    }
]

# Execute the aggregation pipeline
result = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print("Number of documents in the result:", len(result))

# Print the first 10 results
print("First 10 results:")
pprint(result[:10])

Number of documents in the result: 55
First 10 results:
[{'_id': 'Thanet', 'count': 10170},
 {'_id': 'Greenwich', 'count': 7938},
 {'_id': 'Maidstone', 'count': 6417},
 {'_id': 'Newham', 'count': 6399},
 {'_id': 'Swale', 'count': 6174},
 {'_id': 'Chelmsford', 'count': 6120},
 {'_id': 'Medway', 'count': 6048},
 {'_id': 'Bexley', 'count': 5463},
 {'_id': 'Southend-On-Sea', 'count': 5274},
 {'_id': 'Tendring', 'count': 4878}]


In [14]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(result)

# Display the number of rows in the DataFrame
num_rows = len(df)
print(f"Number of rows in the DataFrame: {num_rows}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))


Number of rows in the DataFrame: 55
First 10 rows of the DataFrame:
               _id  count
0           Thanet  10170
1        Greenwich   7938
2        Maidstone   6417
3           Newham   6399
4            Swale   6174
5       Chelmsford   6120
6           Medway   6048
7           Bexley   5463
8  Southend-On-Sea   5274
9         Tendring   4878
